In [1]:
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
from keras.utils import np_utils

from google.colab import drive
import os
import librosa
from librosa.feature import spectral_bandwidth

# Mount Google Drive for access
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:

# Load data from JSON files
with open('/content/drive/MyDrive/My-207/X_train_bandwidth.json', 'r') as file:
    X_train = np.array(json.load(file))

with open('/content/drive/MyDrive/My-207/y_train_bandwidth.json', 'r') as file:
    y_train = np.array(json.load(file))

with open('/content/drive/MyDrive/My-207/X_val_bandwidth.json', 'r') as file:
    X_val = np.array(json.load(file))

with open('/content/drive/MyDrive/My-207/y_val_bandwidth.json', 'r') as file:
    y_val = np.array(json.load(file))

In [4]:
from sklearn.utils import shuffle
# Standardize the feature values (mean=0, std=1) for better performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Encode labels to integers for the model
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.transform(y_val)

# Shuffle the training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Get the current shape of the data
print("Current shape of the data:", X_train.shape)

# Reshape the data to fit the model
num_rows = X_train.shape[1]
num_channels = 1

X_train = X_train.reshape(X_train.shape[0], num_rows, num_channels)
X_val = X_val.reshape(X_val.shape[0], num_rows, num_channels)

print("New shape of the data:", X_train.shape)


Current shape of the data: (4555, 251)
New shape of the data: (4555, 251, 1)


In [5]:

# Construct model
model = Sequential()

# 1st conv layer
model.add(Conv1D(16, 2, activation='relu', input_shape=(num_rows, num_channels)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

# 2nd conv layer
model.add(Conv1D(32, 2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

# 3rd conv layer
model.add(Conv1D(64, 2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

# 4th conv layer
model.add(Conv1D(128, 2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))



In [6]:
# flattening the data for fully connected layers
model.add(GlobalAveragePooling1D())

# output layer
model.add(Dense(len(np.unique(y_train)), activation='softmax'))

# compile model
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='adam')




In [7]:
# train model
history = model.fit(X_train, y_train, batch_size=256, epochs=30, validation_data=(X_val, y_val), verbose=1)


Epoch 1/30
18/18 [==============================] - 7s 155ms/step - loss: 1.0919 - accuracy: 0.3693 - val_loss: 1.0974 - val_accuracy: 0.3339
Epoch 2/30
18/18 [==============================] - 3s 145ms/step - loss: 1.0772 - accuracy: 0.4314 - val_loss: 1.0896 - val_accuracy: 0.3266
Epoch 3/30
18/18 [==============================] - 3s 145ms/step - loss: 1.0544 - accuracy: 0.4648 - val_loss: 1.0805 - val_accuracy: 0.3423
Epoch 4/30
18/18 [==============================] - 4s 238ms/step - loss: 1.0206 - accuracy: 0.4935 - val_loss: 1.0627 - val_accuracy: 0.3882
Epoch 5/30
18/18 [==============================] - 3s 139ms/step - loss: 0.9937 - accuracy: 0.5100 - val_loss: 1.0597 - val_accuracy: 0.4000
Epoch 6/30
18/18 [==============================] - 3s 142ms/step - loss: 0.9735 - accuracy: 0.5260 - val_loss: 1.0775 - val_accuracy: 0.4106
Epoch 7/30
18/18 [==============================] - 2s 137ms/step - loss: 0.9573 - accuracy: 0.5317 - val_loss: 1.0725 - val_accuracy: 0.4291
Epoch 

In [ ]:

# Plotting the accuracy and loss
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()